<img src="./images/DLI_Header.png" style="width: 400px;">



# 3.0 Multi-Node Distributed Training Strategies

In this notebook, we will learn how to run [Megatron-LM](https://github.com/NVIDIA/Megatron-LM) GPT pretraing on multiple nodes.


## The goals

The goals of this notebook are to:
* Run simple multi-node training of Megatron-LM scripts
* Run a hybrid multi-node execution with data, tensor and pipeline parallel distributions


**[3.1 Multi-Node Training Execution of Megatron-LM GPT Pretraining](#1.1-The-hardware-overview)<br>**
**[3.2 Multi-Node Execution with Data Parallelism](#1.1-The-hardware-overview)<br>**
**[3.3 Inter/Intra Node Communications](#1.1-The-hardware-overview)<br>**
**[3.4 Profiling](#1.1-Profiling)<br>**
**[3.5 Exercise: Hybrid Distributed Training Strategy](#1.1-The-hardware-overview)<br>**
**[3.6 Increase The Batchsize / GPU](#1.1-The-hardware-overview)<br>**

### Cancel Previous Running/Pending Jobs

Before moving on, check that no jobs are still running or waiting on the SLURM queue. Let's check the SLURM jobs queue by executing the following cell:

In [1]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


If there are still jobs running or pending, execute the following cell to cancel all the user's jobs using the `scancel` command. 

In [2]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


---
# 3.1 Multi-Node Training Execution of Megatron-LM GPT Pretraining

In the previous notebook, we submitted our jobs in an interactive session after allocating 1 node. 

For multi-node jobs, we need to rely on the SLURM scheduler. By default, multi-node training with Megatron-LM uses the [NVIDIA Collective Communications Library - NCCL](https://developer.nvidia.com/nccl) distributed backend of the [PyTorch distributed launcher](https://pytorch.org/docs/stable/distributed.html).

For 2-Node execution, we will use `SBATCH` scripting. While python execution commands and their arguments remain similar to how it was done for Single Node, we will need additional `SBATCH` arguments for resource allocation. 

Let's start by executing Megatron-LM GPT pretraining on 2 nodes using only Data Parallelism, meaning that the model is copied on the 4 allocated GPUs, each processing different data batches.

# 3.2 Multi-Node Execution with Data Parallelism

In the previous 2-GPU data parallelism execution, the batch size processed by each GPU was 2 (set by `--micro-batch-size`) corresponding to a global batch size of 4 (set by `--global-batch-size`). 

When using 4 GPUs, we can keep the micro batch size per GPUs to 2 and set the global batch size to 8 (MICRO_BATCH_SIZE $ \times $ 4).

Let's have a look at the script before allocating resources and executing it. 

Notice the `SBATCH` arguments for allocating resources. 

In [3]:
# have a look at Megaton-LM GPT pretraining execution on 2 nodes
! cat /dli/code/pretrain_gpt_2Node4GPU.sh

#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=2
GLOBAL_BATCH_SIZE=8    # <--- CHANGED HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS

Now, let's submit the sbatch [script pretrain_gpt_2Node4GPU.sh](./code/pretrain_gpt_2Node4GPU.sh) and check the SLURM queue using the `squeue` command.

In [4]:
# submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU.sh

# Check the SLURM queue
!squeue

Submitted batch job 10
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                10  slurmpar dli_2nod    admin PD       0:00      2 (None)


We can check the GPU usage by running the `nvidia-smi` command on the master lab node. After a few seconds, when the nodes are allocated and the script begins its execution, we should see the GPUs 0,1,2,3 utilized, as shown below. Please notice that if you are running Megatron-LM for the first time in the class, the code will need about 6 minutes to be compiled. Until there, you will not be able to see any GPU activity.


<img src="images/2N_4gpus_utilization.png" width="650"/>

In [5]:
# Check GPU utilization on the master node
! sleep 10
!nvidia-smi

Thu Mar 23 17:40:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Graphics Device     On   | 00000001:00:00.0 Off |                    0 |
| N/A   43C    P0   100W / 300W |  12482MiB / 81251MiB |     98%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  Graphics Device     On   | 00000002:00:00.0 Off |                    0 |
| N/A   

To understand the performance of the Megatron GPT-3 pretraining, we can check the generated [logs](./megatron/logs/log_2Nodes4GPUS.txt) during execution.

Let's first verify the world size of our run. We should see this:
```
using world size: 4, data-parallel-size: 4, tensor-model-parallel size: 1, pipeline-model-parallel size: 1
```

In [6]:
! grep "using world size:" /dli/megatron/logs/log_2Nodes4GPUS.txt

using world size: 4, data-parallel-size: 4, tensor-model-parallel size: 1, pipeline-model-parallel size: 1 


Let's now check the performance of the GPT pretraining on 4 GPUs.

In [7]:
! grep iteration /dli/megatron/logs/log_2Nodes4GPUS.txt

 iteration       10/     100 | consumed samples:           80 | elapsed time per iteration (ms): 457.9 | learning rate: 6.000E-05 | global batch size:     8 | lm loss: 1.048261E+01 | loss scale: 1.0 | grad norm: 3.804 | number of skipped iterations:   0 | number of nan iterations:   0 |
[Rank 0] (after 10 iterations) memory (MB) | allocated: 1907.2294921875 | max allocated: 10582.0205078125 | reserved: 10922.0 | max reserved: 10922.0
 iteration       20/     100 | consumed samples:          160 | elapsed time per iteration (ms): 297.8 | learning rate: 5.997E-05 | global batch size:     8 | lm loss: 9.940797E+00 | loss scale: 1.0 | grad norm: 2.449 | number of skipped iterations:   0 | number of nan iterations:   0 |
 iteration       30/     100 | consumed samples:          240 | elapsed time per iteration (ms): 298.3 | learning rate: 5.990E-05 | global batch size:     8 | lm loss: 9.499748E+00 | loss scale: 1.0 | grad norm: 2.510 | number of skipped iterations:   0 | number of nan iter

From the extract logs, we can see the training performance while using 4 GPUs (2 GPUs per node). 

```
iteration      100/     100 | consumed samples:          800 | elapsed time per iteration (ms): 537.3 | learning rate: 5.822E-05 | global batch size:     8 | lm loss: 7.448950E+00 | loss scale: 1.0 | grad norm: 1.303 | number of skipped iterations:   0 | number of nan iterations:   0 |
```
Compare this with 1 Node executions, and discuss it with the instructor.

---
# 3.3 Inter/Intra Node Communications 

In the previous run, you may have noticed that we added the NCCL variable `NCCL_DEBUG=INFO` in order to output the NCCL debug log traces during training as shown bellow. 

![title](images/nodes_communication.png)


This will allow us to check the type of networking used between GPUs and nodes during the training. 

The direct GPU-to-GPU communication are reposted as `P2P/IPC` while internode communications are reported to be through the `NET/Socket/0`.
In our configuration, direct GPU-to-GPU communication should be used within nodes: GPU0<->GPU1 and GPU2<->GPU3). 

Let's check what NCCL reported in the generated [logs](./megatron/logs/log_2Nodes4GPUS.txt) during the previous execution.

In [8]:
! grep Channel /dli/megatron/logs/log_2Nodes4GPUS.txt | grep slurmnode1

slurmnode1:2181:2348 [0] NCCL INFO Channel 00/02 :    0   1   2   3
slurmnode1:2181:2348 [0] NCCL INFO Channel 01/02 :    0   1   2   3
slurmnode1:2181:2348 [0] NCCL INFO Channel 00 : 3[400000] -> 0[100000] [receive] via NET/Socket/0
slurmnode1:2181:2348 [0] NCCL INFO Channel 01 : 3[400000] -> 0[100000] [receive] via NET/Socket/0
slurmnode1:2182:2349 [1] NCCL INFO Channel 00 : 1[200000] -> 2[300000] [send] via NET/Socket/0
slurmnode1:2181:2348 [0] NCCL INFO Channel 00 : 0[100000] -> 1[200000] via P2P/IPC/read
slurmnode1:2181:2348 [0] NCCL INFO Channel 01 : 0[100000] -> 1[200000] via P2P/IPC/read
slurmnode1:2182:2349 [1] NCCL INFO Channel 01 : 1[200000] -> 2[300000] [send] via NET/Socket/0
slurmnode1:2182:2349 [1] NCCL INFO Channel 00 : 1[200000] -> 0[100000] via P2P/IPC/read
slurmnode1:2182:2349 [1] NCCL INFO Channel 01 : 1[200000] -> 0[100000] via P2P/IPC/read
slurmnode1:2181:2348 [0] NCCL INFO Channel 00 : 2[300000] -> 0[100000] [receive] via NET/Socket/0
slurmnode1:2181:2348 [0] NCC

---
# 3.4 Monitoring and Profiling the Training


So far, monitoring the training runs with Megatron-LM was done via the text log files. However, monitoring training is also possible through tensorboard visualization of the hyper parameters and training/evaluation metrics. In addition, visualizing can be helpful for debugging and optimizing the models during training.

## 3.4.1 Training Metrics Visualization on Tensorboard

<img src="images/tensorboard1.png" width="950"/>

In the previous Megatron-LM runs, we set the arguments for recording Tensorboard events. The graphs of all the previous experiments are available in the folder `megatron/tensorboard/`.

Execute the next cell to create a link to Tensorboard for the browser. Then, click the link to see graphs of experiment metrics saved in the specified `Tensorboard` directory. 

In [9]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

## 3.4.2 Pytorch Profiler with Tensorboard

Several existing profiling tools can be used to investigate bottlenecks during the training and inference processes. This allows us to identify the most expensive operations, issues such as GPU starvation, or unnecessary operations. 

In this class, we will use the [Pytorch Profiler](https://pytorch.org/docs/stable/profiler.html), a tool collecting performance metrics during Pytorch executions. The [Tensorboard-plugin](https://pytorch.org/tutorials/intermediate/tensorboard_profiler_tutorial.html) with the PyTorch Profiler provides a visualization of each GPU process, with several measures of operations running on each GPU, whether using hardware accelerator (TensorCores). It also provides recommendations to improve the process.

We will trace only the execution of the GPU_0, monitoring operations during 2 training steps. We specified the profiling by setting `profile-execution` to *True* and providing the `profile-name` to *baseline*.

For the previous run, profiling is available on the Tensorboard link at the `pytorch_profiler` tab.
<img src="images/profiling1.png" width="900"/>

In case you already closed the Tensorboard page, you can re-generate the link by executing the next cell. Click the link to open Tensorboard and then, go to the `PYTORCH_PROFILER` tab.


In [10]:
%%js
const href = window.location.hostname +'/tensorboard/';
let a = document.createElement('a');
let link = document.createTextNode('Open Tensorboard!');
a.appendChild(link);
a.href = "http://" + href;
a.style.color = "navy"
a.target = "_blank"
element.append(a);

<IPython.core.display.Javascript object>

The profiler homepage shows an overview of tracing. 


### The Profiling Overview
Several panels are shown on the homepage:
- The GPU Summary: shows the GPU configuration and utilization. In our previous run, no TensorCores are used as we have not enabled Mixed Precision training (will see this in the next notebook). 

- The Step Time Breakdown: shows the execution time spent in each operation. In our previous run,  48.5% of the step time is in communication and 41.7% in Kernel execution time on GPU 

- Performance Recommendation: Providing recommendations on how to improve the process. There are a few recommendations for our previous run, such as reducing communication cost by using Gradient Accumulation or increasing the batch size. The profiler suggests also enabling Automatic Mixed Precision to speedup Kernels. In the next section, let us first start by increasing the batch size and see its impact in the process (speedup and memory consumption).

### The Memory View
Let's have a look at the Memory View showing the memory allocation and deallocation of our run. We can zoom into the memory curve to see the related memory events.

<img src="images/profiling4.png" width="750"/>

The memory allocation followed by deallocation corresponds to the forward and backward pass. This process is traced twice as we traced 2 training steps. We can also see that a peak of ~10GB of the GPU device memory is used during the training step. We can increase the memory usage by increasing the model or the batch size.


Great!\
Before moving on, let's release some disk space by deleting the unnecessary checkpoints generated by the previous execution.

In [11]:
# Clean the checkpoints
! rm /dli/megatron/checkpoints/* -r 

---
# 3.5 Increase The Batchsize / GPU

**Special Warning:** In this section, Out Of Memory (OOM) issues are expected! No problem, we will see a few solutions addressing this problem in the next notebook.


The size of the current GPT model fits into the GPU memory and thus does not require model distribution (tensor or pipeline parallelism). \
Using only data parallelism, we can improve the training throughput (number of sequences processed per second) by increasing the batch size processed per GPU until reaching the maximum batch size that fits on GPU memory.\
Let's check this by doubling the data processed by each GPU. When using 4 GPUs with only data parallelism (DP_SIZE $= 4$), by increasing the micro batch size per GPUs from 2 to 4, the global batch size should be MICRO_BATCH_SIZE $\times$ TP_SIZE $= 16$.

Let's prepare the training script for this scenario by running the following cell.

In [22]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=1
PP_SIZE=1 

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=32      # <--- CHANGED HERE
GLOBAL_BATCH_SIZE=128    # <--- CHANGED HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS_DP_4_MBS_4"


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
              "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            $OPTIMIZER_ARGS \
        "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            --profile-execution True \
            --profile-name DP_4_MBS_4 \
            "

export LAUNCHER="python -u -m torch.distributed.launch \
             --nproc_per_node $GPUS_PER_NODE \
             --nnodes $NNODES \
             --master_addr $MASTER_ADDR \
             --master_port $MASTER_PORT \
             "

export CMD=" \
             /dli/megatron/Megatron-LM/pretrain_gpt.py \
             --tensor-model-parallel-size $TP_SIZE \
             --pipeline-model-parallel-size $PP_SIZE \
             $GPT_ARGS \
             $OUTPUT_ARGS \
             --save $CHECKPOINT_PATH \
             --data-path $DATA_PATH \
             --data-impl mmap \
             --split 949,50,1 \
             --distributed-backend nccl \
           "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO  $LAUNCHER --node_rank $SLURM_PROCID $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh


Now, let's submit the previous sbatch script [pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh](./code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh) and check the SLURM queue using the squeue command.

In [17]:
# submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU_DP_4_MBS_4.sh

# Check the SLURM queue
!squeue

Submitted batch job 12
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
                12  slurmpar dli_2nod    admin PD       0:00      2 (None)


Let's now check the Megatron GPT-3 pretraining performance by looking at the generated [logs](./megatron/logs/log_2Nodes4GPUS_hybrid_solution.txt) during the execution.

In [18]:
! grep iteration /dli/megatron/logs/log_2Nodes4GPUS_DP_4_MBS_4.txt

[Rank 0] (after 10 iterations) memory (MB) | allocated: 1905.9794921875 | max allocated: 19084.1767578125 | reserved: 19876.0 | max reserved: 19876.0
 iteration       10/     100 | consumed samples:          160 | elapsed time per iteration (ms): 549.2 | learning rate: 6.000E-05 | global batch size:    16 | lm loss: 1.045963E+01 | loss scale: 1.0 | grad norm: 2.574 | number of skipped iterations:   0 | number of nan iterations:   0 |
 iteration       20/     100 | consumed samples:          320 | elapsed time per iteration (ms): 407.1 | learning rate: 5.997E-05 | global batch size:    16 | lm loss: 9.929971E+00 | loss scale: 1.0 | grad norm: 2.603 | number of skipped iterations:   0 | number of nan iterations:   0 |
 iteration       30/     100 | consumed samples:          480 | elapsed time per iteration (ms): 405.8 | learning rate: 5.990E-05 | global batch size:    16 | lm loss: 9.441203E+00 | loss scale: 1.0 | grad norm: 2.433 | number of skipped iterations:   0 | number of nan iter


No elapsed time per iteration is shown. **What just happened?!** 

Are we saturating the GPU memory? Run the cell bellow to search for RuntimeError in our logs. You should see:
```
RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 15.78 GiB total capacity; 13.90 GiB already allocated; 302.00 MiB free; 14.43 GiB reserved in total by PyTorch)
RuntimeError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 1; 15.78 GiB total capacity; 13.90 GiB already allocated; 302.00 MiB free; 14.43 GiB reserved in total by PyTorch)
```

In [19]:
! grep "RuntimeError" /dli/megatron/logs/log_2Nodes4GPUS_DP_4_MBS_4.txt

RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 1; 79.35 GiB total capacity; 76.80 GiB already allocated; 915.19 MiB free; 76.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 1; 79.35 GiB total capacity; 76.80 GiB already allocated; 915.19 MiB free; 76.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
RuntimeError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 79.35 GiB total capacity; 76.80 GiB already allocated; 915.19 MiB free; 76.95 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for


Indeed, we should see `CUDA out of memory` errors which means that the GPU memory cannot handle training this transformer model size with the specified arguments.

Doubling the amount of data processed per GPU is too much for the 16G of memory.  

In the next lab, we will focus on how to address this issue by reducing the model's memory footprint.

Great, before moving on, let's release some disk space by deleting the unnecessary checkpoints generated by the previous execution.

In [20]:
# Clean the checkpoints and logs directory
! rm -rf /dli/megatron/checkpoints/* 

#### ---
# 3.6 Exercise: Hybrid Distributed Training Strategy 


Let's configure a new Megatron-LM GPT pretraining execution on 2 nodes (4 GPUs) with both tensor and pipeline parallel by modifying the "FIXME" in the following cell. 

To use tensor and pipeline parallel, we can keep distributed the model using in 2 dimensions: 2 GPUs for tensor parallelism and 2 GPUs for pipeline parallelism. Thus, no more resources will remain for the data parallel. In this case, the `GLOBAL_BATCH_SIZE` should be downgraded to the same size as the `MICRO_BATCH_SIZE`.

If you get stuck, view the [solution](solutions/ex3.4.ipynb) for a hint.


In [23]:
%%writefile /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh
#!/bin/bash
#SBATCH --job-name=dli_2nodes
#SBATCH --nodes=2
#SBATCH --ntasks-per-node=1       
#SBATCH --cpus-per-task=32 ### Number of threads per task (OMP threads)
#SBATCH -o /dli/megatron/logs/%j.out
#SBATCH -e /dli/megatron/logs/%j.err

set -x -e

export OMP_NUM_THREADS=$SLURM_CPUS_PER_TASK

# Distributed training args
NNODES=2
GPUS_PER_NODE=2
TP_SIZE=2        # <--- CHANGE HERE
PP_SIZE=2         # <--- CHANGE HERE

# SLURM args
MASTER_ADDR=$(scontrol show hostnames $SLURM_JOB_NODELIST | head -n 1)
MASTER_PORT=6000

# Distributed training 
MICRO_BATCH_SIZE=4         # <--- CHANGE HERE
GLOBAL_BATCH_SIZE=4        # <--- CHANGE HERE

# Model architecture 
NLAYERS=12
NHIDDEN=768
NHEADS=32
SEQ_LEN=1024
VOCAB_SIZE=50257

# Data Paths
DATA_OUTPUT_PATH=/dli/megatron/checkpoints/test
CHECKPOINT_PATH=/dli/megatron/checkpoints
TENSORBOARD_PATH=/dli/megatron/tensorboard
LOGS_PATH=/dli/megatron/logs
VOCAB_FILE=/dli/data/GPT-2_assets/gpt2-vocab.json
MERGE_FILE=/dli/data/GPT-2_assets/gpt2-merges.txt
DATA_PATH=/dli/data/GPT-2_assets/my-gpt2_text_document

NAME="log_2Nodes4GPUS_hybrid"


OPTIMIZER_ARGS=" \
            --optimizer adam \
            --adam-beta1 0.9 \
            --adam-beta2 0.95 \
            --adam-eps 1e-8 \
            --lr 6e-5 \
            --min-lr 6e-6 \
            --lr-decay-style cosine \
            --lr-decay-iters 800 \
            --lr-warmup-fraction .01 \
            --clip-grad 1.0 \
            --weight-decay 1e-1 \
            --exit-duration-in-mins 1190 \
              "

GPT_ARGS=" \
            --num-layers $NLAYERS \
            --hidden-size $NHIDDEN \
            --num-attention-heads $NHEADS \
            --seq-length $SEQ_LEN \
            --max-position-embeddings $SEQ_LEN \
            --micro-batch-size $MICRO_BATCH_SIZE \
            --global-batch-size $GLOBAL_BATCH_SIZE \
            --train-iters 100 \
            --vocab-file $VOCAB_FILE \
            --merge-file $MERGE_FILE \
            --init-method-std 0.006 \
            $OPTIMIZER_ARGS \
        "

OUTPUT_ARGS=" \
            --log-interval 10 \
            --save-interval 300 \
            --eval-interval 1000 \
            --eval-iters 10 \
            --tensorboard-dir $TENSORBOARD_PATH \
            --tensorboard-queue-size 1 \
            --log-timers-to-tensorboard \
            --log-batch-size-to-tensorboard \
            --log-validation-ppl-to-tensorboard \
            --profile-execution True \
            --profile-name DP_PP \
            "

export LAUNCHER="python -u -m torch.distributed.launch \
             --nproc_per_node $GPUS_PER_NODE \
             --nnodes $NNODES \
             --master_addr $MASTER_ADDR \
             --master_port $MASTER_PORT \
             "

export CMD=" \
             /dli/megatron/Megatron-LM/pretrain_gpt.py \
             --tensor-model-parallel-size $TP_SIZE \
             --pipeline-model-parallel-size $PP_SIZE \
             $GPT_ARGS \
             $OUTPUT_ARGS \
             --save $CHECKPOINT_PATH \
             --data-path $DATA_PATH \
             --data-impl mmap \
             --split 949,50,1 \
             --distributed-backend nccl \
           "

clear; srun --jobid $SLURM_JOBID bash -c 'NCCL_DEBUG=INFO  $LAUNCHER --node_rank $SLURM_PROCID $CMD' 2>&1 | tee -a $LOGS_PATH/$NAME.txt

Overwriting /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh


Now, let's submit the previous sbatch script [pretrain_gpt_2Node4GPU_hybrid.sh](/dli/code/pretrain_gpt_2Node4GPU_hybrid.sh) for hybrid multi-node run and check the SLURM queue using the `squeue` command.

In [ ]:
# Submit the 2 nodes jobs
! sbatch /dli/code/pretrain_gpt_2Node4GPU_hybrid.sh

# Check the SLURM queue
!squeue


To understand the performance of the Megatron GPT-3 pretraining, we can check the generated logs during the execution.

Let's first look at the generated [logs](/dli/megatron/logs/log_2Nodes4GPUS_hybrid.txt) and check world size of our executed hybrid run. We should see this:

```
using world size: 4, data-parallel-size: 1, tensor-model-parallel size: 2, pipeline-model-parallel size: 2
```


In [ ]:
! grep "using world size:" /dli/megatron/logs/log_2Nodes4GPUS_hybrid.txt

Let's check the training performance and GPU0 memory allocation and deallocation of our run. You should see a graph similar to the following: 
<img src="images/profiling_hybrid.png" width="950"/>

What is the constant time step between the forward and backward pass? Discuss with the instructor.

In [ ]:
! grep iteration /dli/megatron/logs/log_2Nodes4GPUS_hybrid.txt

Great, before moving on, let's release some disk space by deleting the unnecessary checkpoints generated by the previous execution.


In [ ]:
# Clean the checkpoints 
! rm -rf /dli/megatron/checkpoints/* 

---
<h2 style="color:green;">Congratulations!</h2>

Before moving on, we need to make sure no jobs are still running or waiting in the queue. 

In [24]:
# Check the SLURM jobs queue 
!squeue

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


If there are still jobs running or pending, execute the following cell to cancel all the jobs using the `scancel` command. 

In [ ]:
# Cancel admin user jobs
! scancel -u $USER

# Check again the SLURM jobs queue (should be either empty, or the status TS column should be CG)
! squeue

Next, you will see how to optimize the GPT pretraining using techniques such as Mixed Precision, Gradient Accumulation and Activation Checkpointing. Move on to [004_GPT_LM_pretrainings_optimizations.ipynb](04_GPT_LM_pretrainings_optimizations.ipynb).